# Proyecto Grupo 36 - Comedor Social

Esta es la implementación en Gurobi del proyecto presentado por el grupo 36.

Dado N días, consideramos los siguientes parámetros:

* $g_i$: cantidad de gente que llega el día $i$.
* $c_a$: costo del alimento $a$.
* $cs$: sueldo fijo del trabajor remunerado. (mensual)
* $dm_i$: donaciones monetarias recibidas el día $i$ (total, de todas las fuentes).
* $dc_{ia}$: cantidad de alimento $a$ que fue recibido como donación el día $i$.
* $e$: entrada que se cobra para comer (200 CLP)
* $d_a$: duración del alimento $a$
* $ap_a$: $\begin{cases}
                    1 & \text{si alimento $a$ es rico en proteinas.} \\
                    0 & \text{otro caso.}
                \end{cases}$
* $ac_a$: $\begin{cases}
                    1 & \text{si alimento $a$ es rico en carbohidratos.} \\
                    0 & \text{otro caso.}
                \end{cases}$
* $av_a$: $\begin{cases}
                    1 & \text{si alimento $a$ es una verdura.} \\
                    0 & \text{otro caso.}
                \end{cases}$
* $af_a$: $\begin{cases}
                    1 & \text{si alimento $a$ es una fruta.} \\
                    0 & \text{otro caso.}
                \end{cases}$
* $v_a$: volumen que ocupa el alimento $a$.
* $vmax$: volumen máximo de alimentos que se pueden almacenar en despensa.
* $amax_i$: cantidad máxima de alimentos que los cocineros están dispuestos a incluir en la confección de un plato el día $i$.

Estos parámetros se encuentran en el archivo `data.json`.

In [1]:
from gurobipy import *
import json

from IPython.display import HTML, display
import tabulate

# Create a new model
m = Model("Comedor social")

# parametros

with open('data.json') as f:
    data = json.loads(f.read())

dias = [['Dia', 'Alimentos Máximos', 'Donaciones Diarias', 'Visitantes']]
dias_temp = []
alimentos = [['Alimento', 'Volumen Alimento', 'Duracion Alimento', 'Costo Alimento', 'Verdura', 'Fruta', 'Proteina', 'Carbohidrato']]
alimentos_temp = []

donaciones = [['Alimento'] + [f'Día {n}' for n in range(data['dias'])]]
donaciones_temp = []

    
for k, i in data.items():
    if isinstance(i, (int, float)):
        print(k.capitalize() + ':', i)
    elif isinstance(i, list):
        if k in ('amax', 'donaciones_monetarias', 'visitas'):
            dias_temp.append(i)
        elif k != 'cantidad_alimento':
            alimentos_temp.append(i)
        else:
            donaciones_temp.append(i)

dias.extend(list(map(lambda x: [x[0]] + list(x[1]), ((x, n) for x, n in enumerate(zip(*dias_temp))))))
alimentos.extend(list(map(lambda x: [x[0]] + list(x[1]), ((x, n) for x, n in enumerate(zip(*alimentos_temp))))))
donaciones.extend(list(map(lambda x: [x[0]] + x[1][0], ((x, n) for x, n in enumerate(zip(*donaciones_temp))))))
        
display(HTML(tabulate.tabulate(dias, tablefmt='html')))
display(HTML(tabulate.tabulate(alimentos, tablefmt='html')))
display(HTML(tabulate.tabulate(donaciones, tablefmt='html')))

Academic license - for non-commercial use only
Dias: 31
Alimentos: 50
Sueldo_fijo: 250000
Vol_max: 500
Entrada: 200


Dia,Alimentos Máximos,Donaciones Diarias,Visitantes
0,5,24956,82
1,4,26797,80
2,7,22404,57
3,4,28000,79
4,7,17985,72
5,4,29468,30
6,5,20380,89
7,4,24620,71
8,7,22009,108
9,7,34700,30


Alimento,Volumen Alimento,Duracion Alimento,Costo Alimento,Verdura,Fruta,Proteina,Carbohidrato
0,1,54,1036,0,0,0,1
1,1,53,820,0,0,0,1
2,1,4,2079,0,1,0,0
3,1,6,330,0,0,1,0
4,1,6,1324,0,0,1,0
5,1,4,1189,0,1,0,0
6,1,4,891,1,0,0,0
7,1,3,300,0,0,1,0
8,1,59,1300,0,0,0,1
9,1,4,741,0,1,0,0


Alimento,Día 0,Día 1,Día 2,Día 3,Día 4,Día 5,Día 6,Día 7,Día 8,Día 9,Día 10,Día 11,Día 12,Día 13,Día 14,Día 15,Día 16,Día 17,Día 18,Día 19,Día 20,Día 21,Día 22,Día 23,Día 24,Día 25,Día 26,Día 27,Día 28,Día 29,Día 30
0,4,5,4,4,4,4,2,5,1,7,4,0,0,2,1,0,2,0,1,7,1,4,3,3,4,0,0,7,4,4,1
1,5,3,5,0,5,7,5,3,4,7,3,1,4,5,5,4,3,8,3,6,2,1,3,1,4,6,1,3,2,2,3
2,3,4,5,7,5,1,3,5,4,3,1,5,6,8,4,1,1,3,2,7,2,3,7,4,0,3,2,3,6,5,0
3,4,2,0,3,1,4,4,2,3,1,4,1,1,5,0,5,4,1,2,1,1,3,2,2,5,2,5,3,0,5,9
4,5,5,4,3,1,1,2,3,2,0,3,2,2,3,4,4,6,5,2,0,3,3,5,1,3,6,7,5,2,4,2
5,3,4,4,1,4,1,4,3,0,3,4,4,5,5,0,1,3,2,2,5,5,4,9,3,6,4,1,2,3,4,2
6,5,2,8,5,1,4,2,4,5,1,1,4,3,2,3,3,4,4,2,3,5,6,7,2,1,1,1,3,4,6,4
7,5,2,4,5,7,3,1,1,2,4,3,3,4,0,2,3,0,5,7,1,4,5,3,7,6,4,3,5,0,3,6
8,3,5,9,0,5,5,6,2,2,6,5,4,4,0,4,4,4,5,2,2,7,6,3,5,3,1,3,2,4,2,6
9,3,6,5,7,4,6,2,5,4,6,2,3,0,3,6,4,5,5,4,4,0,2,5,3,2,2,2,3,3,4,4


## Variables
* $x_{ai}$: cantidad de alimento $a$ que se compra en el día $i$.
* $y_{ai}$ : cantidad de alimento $a$ que se utiliza en el día $i$
* $I_{ai}$: cantidad del alimento $a$ que se almacena el día $i$.
* $z_{i}$: dinero extra (no donaciones) usadas en el día $i$.
* $s_{i}$: dinero sobrante en el día $i$.

In [2]:
# Create variables
X = [[m.addVar(vtype=GRB.INTEGER, name=f"Alimento {a} comprado dia {i}") for a in range(data['alimentos'])] for i in range(data['dias'])]
Y = [[m.addVar(vtype=GRB.INTEGER, name=f"Alimento {a} usado dia {i}") for a in range(data['alimentos'])] for i in range(data['dias'])]
I = [[m.addVar(vtype=GRB.INTEGER, name=f"Alimento {a} almacenado dia {i}") for a in range(data['alimentos'])] for i in range(data['dias'])]
extra = [m.addVar(vtype=GRB.INTEGER, name=f"Dinero extra dia {i}") for i in range(data['dias'])]
surplus = [m.addVar(vtype=GRB.INTEGER, name=f"Surplus el día {i}") for i in range(data['dias'] + 1)]

## Función objetivo
$$\text{min} \sum_{i \in D} z_i$$

In [3]:
# Set objective
m.setObjective(sum(extra[i] for i in range(data['dias'])), GRB.MINIMIZE)

# Restricciones

* Límite de gastos: el gasto diario de dinero no puede superar lo obtenido en donaciones ese día más lo que haya sobrado (lo obtenido menos lo gastado) de los días anteriores. Se incluye el costo diario que significa tener un trabajador remunerado.
    $$\sum_{a \in A} c_a x_{ai} + \frac{cs}{30.5} \leq z_i + dm_i + g_i e + s_{i - 1}   \quad\quad\quad     \forall i \in D$$
* El surplus es el dinero restante de cada día:
    $$ s_{i} = z_i + dm_i + g_i e + s_{i - 1} - (\sum_{a \in A} c_a x_{ai} + \frac{cs}{30.5}) \quad\quad\quad \forall i \in D$$
* No hay dinero extra al comienzo de la optimización:
    $$ s_{-1} = 0$$
* No se puede usar más de lo que se tiene
    $$y_{ai} \leq x_{ai} + I_{a(i-1)}     \quad\quad\quad     \forall a \in A, \forall i \in D$$
* Inventario inicial
    $$I_{a0} = 0     \quad\quad\quad     \forall a \in A$$
* Límite de almacenamiento
    $$\sum_{a \in A} I_{ai} v_a \leq vmax     \quad\quad\quad     \forall i \in D$$
* Una proteina mínimo por comida
    $$ \sum_{a \in A} y_{ai}  ap_a \geq g_i \quad\quad\quad \forall i \in D $$
* Un carbohidrato mínimo por comida
    $$ \sum_{a \in A} y_{ai}  ac_a \geq g_i \quad\quad\quad \forall i \in D $$
* Una verdura mínimo por comida
    $$ \sum_{a \in A} y_{ai}  av_a \geq g_i \quad\quad\quad \forall i \in D $$
* Una fruta mínimo por comida
    $$ \sum_{a \in A} y_{ai}  af_a \geq g_i \quad\quad\quad \forall i \in D $$
* Máximo de alimentos por plato de comida
    $$ \sum_{a \in A} (y_{ai} af_a + y_{ai} av_a + y_{ai} ac_a + y_{ai} ap_a) \leq amax_i g_i \quad\quad\quad \forall i \in D$$
* Caducidad de los Alimentos: para cada día y cada alimento, la totalidad de alimentos de cierto tipo recibidos (ya sea por compra o por donación) debe haberse utilizado en tantos días en el futuro como la caducidad de ese alimento. Como los alimentos más viejos siempre se utilizan primero, esto significa que para la fecha que los alimentos vencerian estos ya deben haber sido utilizados. La condición también se cumple en días que no ha habido compra o donación, siempre y cuando se cumpla en el día de la última compra/donación.
    $$ \sum_{j=1}^{i} (dc_{ja} + x_{aj}) \leq \sum_{j=1}^{i + d_a} y_{ja} \quad\quad\quad \forall a \in A, \forall i \in D$$
* Naturaleza de variables
    $$x_{ai} \geq 0     \quad\quad\quad     \forall a \in A, \forall i \in D$$
    $$y_{ai} \geq 0     \quad\quad\quad     \forall a \in A, \forall i \in D$$
    $$I_{ai} \geq 0     \quad\quad\quad     \forall a \in A, \forall i \in D$$
    $$s_{i}  \geq 0     \quad\quad\quad     \forall i \in D$$
    $$x_{ai}, y_{ai}, I_{ai} \in \mathbb{N}\cup\{0\}     \quad\quad\quad     \forall a \in A, \forall i \in D$$

In [4]:
# Add constraint
m.addConstr(surplus[0] == 0, f'Surplus inicial')

for a in range(data['alimentos']):
    m.addConstr(I[0][a] == 0, f"Inventario inicial del alimento {a}")

for i in range(data['dias']):
    m.addConstr(sum(data['costo_alimento'][a] * X[i][a] for a in range(data['alimentos'])) + (data['sueldo_fijo'] / 30.5) <= extra[i] + data['donaciones_monetarias'][i] + data['visitas'][i] * data['entrada'] + surplus[i],
                f"Límite de gastos en el día {i}")
    m.addConstr(surplus[i + 1] <= extra[i] + data['donaciones_monetarias'][i] + data['visitas'][i] * data['entrada'] + surplus[i] - (sum(data['costo_alimento'][a] * X[i][a] for a in range(data['alimentos'])) + (data['sueldo_fijo'] / 30.5)),
                f"Surplus en el día {i}")
    m.addConstr(sum(I[i][a] * data['volumen_alimentos'][a] for a in range(data['alimentos'])) <= data['vol_max'],
                f"Límite de almacenamiento día {i}")
    m.addConstr(sum(Y[i][a] * data['proteina'][a] for a in range(data['alimentos'])) >= data['visitas'][i],
                f"Una proteina mínimo por comida día {i}")
    m.addConstr(sum(Y[i][a] * data['carbohidrato'][a] for a in range(data['alimentos'])) >= data['visitas'][i],
                f"Un carbohidrato mínimo por comida día {i}")
    m.addConstr(sum(Y[i][a] * data['verdura'][a] for a in range(data['alimentos'])) >= data['visitas'][i],
                f"Una verdura mínimo por comida día {i}")
    m.addConstr(sum(Y[i][a] * data['fruta'][a] for a in range(data['alimentos'])) >= data['visitas'][i],
                f"Una fruta mínimo por comida día {i}")
#    m.addConstr(sum(Y[i][a] * (data['proteina'][a] + data['carbohidrato'][a] + data['verdura'][a] + data['fruta'][a]) for a in range(data['alimentos'])) <= data['amax'][i] * data['visitas'][i],
#                f"Máximo de alimentos por plato de comida día {i}")
    m.addConstr(extra[i] >= 0, f"Naturaleza de Z en dia {i}")
    m.addConstr(surplus[i] >= 0, f"Naturaleza de s en dia {i}")
    for a in range(data['alimentos']):
        m.addConstr(Y[i][a] <= X[i][a] + I[max(0, i - 1)][a] + data['cantidad_alimento'][a][i],
                f"No se puede usar más de lo que se tiene ")
        m.addConstr(I[i][a] <= X[i][a] + (I[i - 1][a] if i else 0) + data['cantidad_alimento'][a][i] - Y[i][a],
                   f'No se puede guardar más de lo que no se usa')
#        m.addConstr(sum(data['cantidad_alimento'][a][j] + X[j][a] for j in range(0, i + 1)) <= sum(Y[k][a] for k in range(0, min(data['dias'] - 1, i + data['duracion_alimentos'][a]))),
#                f"Caducidad de los Alimentos")
        m.addConstr(X[i][a] >= 0, f"Naturaleza de X en dia {i} y alimento {a}")
        m.addConstr(Y[i][a] >= 0, f"Naturaleza de Y en dia {i} y alimento {a}")
        m.addConstr(I[i][a] >= 0, f"Naturaleza de I en dia {i} y alimento {a}")
    

# Resultado

In [5]:
m.optimize()

comprado = []
usado = []
guardado = []
extra = []

for v in m.getVars():
    if 'comprado' in v.varname:
        comprado.append(v)
    elif 'usado' in v.varname:
        usado.append(v)
    elif 'guardado' in v.varname:
        guardado.append(v)
    elif 'extra' in v.varname:
        extra.append(v)
    elif 'Surplus' in v.varname:
        surplus.append(v)

comprado_temp = [['Alimento'] + [f'Día {a}' for a in range(data['dias'])]]
comprado_temp.extend([[f'Alimento {a}'] + [0 for a in range(data['dias'])] for a in range(data['alimentos'])])

usado_temp = [['Alimento'] + [f'Día {a}' for a in range(data['dias'])]]
usado_temp.extend([[f'Alimento {a}'] + [0 for a in range(data['dias'])] for a in range(data['alimentos'])])

guardado_temp = [['Alimento'] + [f'Día {a}' for a in range(data['dias'])]]
guardado_temp.extend([[f'Alimento {a}'] + [0 for a in range(data['dias'])] for a in range(data['alimentos'])])

extra_temp = [['Día', 'Dinero Extra Usado', 'Surplus del día']]

for i in comprado:
    n = i.varname
    dia = int(''.join(x for x in n[-4:] if x.isdecimal()))
    alimento = int(''.join(x for x in n[-20:-9] if x.isdecimal()))
    x = int(i.X)
    comprado_temp[alimento + 1][dia + 1] = f'<b>{x}</b>' if x else 0

print('Alimentos Comprados:')
display(HTML(tabulate.tabulate(comprado_temp, tablefmt='html')))

for i in usado:
    n = i.varname
    dia = int(''.join(x for x in n[-4:] if x.isdecimal()))
    alimento = int(''.join(x for x in n[-20:-9] if x.isdecimal()))
    x = int(i.X)
    usado_temp[alimento + 1][dia + 1] = f'<b>{x}</b>' if x else 0

print('Alimentos Usados:')
display(HTML(tabulate.tabulate(usado_temp, tablefmt='html')))

for i in guardado:
    n = i.varname
    dia = int(''.join(x for x in n[-4:] if x.isdecimal()))
    alimento = int(''.join(x for x in n[-20:-9] if x.isdecimal()))
    x = int(i.X)
    guardado_temp[alimento + 1][dia + 1] = f'<b>{x}</b>' if x else 0

for i in extra:
    n = i.varname
    dia = int(''.join(x for x in n[-4:] if x.isdecimal()))
    x = int(i.X)
    extra_temp.append([f'Día {dia}', f'<b>{x}</b>' if x else 0])

for i in surplus:
    n = i.varname
    dia = int(''.join(x for x in n[-4:] if x.isdecimal()))
    x = int(i.X)
    if len(extra_temp[dia]) == 2 and dia <= data['dias']:
        extra_temp[dia].append(f'<b>{x}</b>' if x else 0)

print('Alimentos Guardados:')
display(HTML(tabulate.tabulate(guardado_temp, tablefmt='html')))

print('Dinero Diario Extra:')
display(HTML(tabulate.tabulate(extra_temp, tablefmt='html')))

print('Gasto total de dinero: %g' % m.objVal)

Optimize a model with 8080 rows, 4713 columns and 21918 nonzeros
Variable types: 0 continuous, 4713 integer (0 binary)
Coefficient statistics:
  Matrix range     [1e+00, 3e+03]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 4e+04]
Found heuristic solution: objective 8044932.0000
Presolve removed 6446 rows and 256 columns
Presolve time: 0.02s
Presolved: 1634 rows, 4457 columns, 10333 nonzeros
Found heuristic solution: objective 7629858.0000
Variable types: 0 continuous, 4457 integer (0 binary)

Root relaxation: objective 4.119550e+05, 3335 iterations, 0.04 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 411955.000    0   68 7629858.00 411955.000  94.6%     -    0s
H    0     0                    411955.00000 411955.000  0.00%     -    0s

Explored 1 nodes (3825 simplex iterations) in 0.17 seconds
Thread count was 4 (

Alimento,Día 0,Día 1,Día 2,Día 3,Día 4,Día 5,Día 6,Día 7,Día 8,Día 9,Día 10,Día 11,Día 12,Día 13,Día 14,Día 15,Día 16,Día 17,Día 18,Día 19,Día 20,Día 21,Día 22,Día 23,Día 24,Día 25,Día 26,Día 27,Día 28,Día 29,Día 30
Alimento 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Alimento 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Alimento 2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Alimento 3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,10,0,0,0
Alimento 4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Alimento 5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Alimento 6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Alimento 7,30,45,0,0,0,5,78,8,0,0,0,0,0,0,0,20,0,0,0,0,0,0,0,0,0,0,92,0,0,0,0
Alimento 8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Alimento 9,55,44,27,38,219,0,0,0,0,2,120,0,68,94,13,0,46,39,0,33,11,25,0,44,11,0,76,0,14,61,24


Alimentos Usados:


Alimento,Día 0,Día 1,Día 2,Día 3,Día 4,Día 5,Día 6,Día 7,Día 8,Día 9,Día 10,Día 11,Día 12,Día 13,Día 14,Día 15,Día 16,Día 17,Día 18,Día 19,Día 20,Día 21,Día 22,Día 23,Día 24,Día 25,Día 26,Día 27,Día 28,Día 29,Día 30
Alimento 0,4,5,4,4,4,0,6,0,6,7,4,0,0,2,1,0,2,0,1,7,0,5,0,6,4,0,0,0,0,15,1
Alimento 1,5,3,5,0,5,7,5,3,4,0,10,0,5,5,3,6,3,0,11,6,0,1,0,4,0,6,5,0,5,2,3
Alimento 2,3,4,5,7,0,0,9,5,4,3,0,0,0,0,24,0,2,0,5,2,7,3,7,0,4,0,0,8,0,11,0
Alimento 3,4,2,0,3,1,4,4,2,3,0,5,1,0,6,0,0,9,1,2,1,0,4,2,0,7,0,7,13,0,3,11
Alimento 4,5,5,4,3,1,1,0,5,0,0,5,2,0,5,4,2,8,5,2,0,0,6,5,0,4,0,13,0,0,11,2
Alimento 5,3,4,4,1,0,0,0,12,0,3,4,0,0,0,5,10,0,0,7,0,0,14,0,0,18,0,5,0,0,0,11
Alimento 6,5,2,8,0,6,3,3,4,5,1,1,0,0,9,0,6,4,4,2,3,0,11,0,0,0,11,1,3,4,4,6
Alimento 7,35,47,4,5,7,8,79,0,0,0,18,3,4,0,0,25,0,5,7,1,0,9,3,7,6,4,42,0,24,37,6
Alimento 8,3,5,9,0,5,0,0,0,15,6,5,0,0,8,4,4,4,5,2,2,0,13,0,0,8,0,0,0,0,12,6
Alimento 9,58,50,32,45,72,30,73,0,65,8,73,52,68,90,26,0,55,0,48,37,0,38,1,27,36,0,80,3,17,66,28


Alimentos Guardados:


Alimento,Día 0,Día 1,Día 2,Día 3,Día 4,Día 5,Día 6,Día 7,Día 8,Día 9,Día 10,Día 11,Día 12,Día 13,Día 14,Día 15,Día 16,Día 17,Día 18,Día 19,Día 20,Día 21,Día 22,Día 23,Día 24,Día 25,Día 26,Día 27,Día 28,Día 29,Día 30
Alimento 0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Alimento 1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Alimento 2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Alimento 3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Alimento 4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Alimento 5,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Alimento 6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Alimento 7,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Alimento 8,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
Alimento 9,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


Dinero Diario Extra:


Día,Dinero Extra Usado,Surplus del día
Día 0,35496,0
Día 1,217550,188646
Día 2,0,184946
Día 3,0,138091
Día 4,0,0
Día 5,0,9871
Día 6,53151,57305
Día 7,0,57328
Día 8,0,58240
Día 9,0,89261


Gasto total de dinero: 411955
